## homework 3-3: Data Research

### load data

In [ ]:
%pip install pandas matplotlib numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

Please make sure folder contains:
- dachang_rent.csv
- dachang_sell.csv
- majiaoqiao_rent.csv
- majiaoqiao_sell.csv
- yanjiao_rent.csv
- yanjiao_sell.csv
- yizhuang_rent.csv
- yizhuang_sell.csv

Though get_file_name(data_type,place) function can handle slightly different file name, it is still recommended to do so.

In [ ]:
place = ["dachang","yanjiao","majiaoqiao","yizhuang"]
data_type = ["sell","rent"]

def get_file_name(data_type,place):
    """
    This function is used to match specific fiile by matching sub-string, 
    so that if group member offer different file_name, this code can still match file.
    """
    try:
        all_files = os.listdir()
    except Exception as e:
        print(f"Error reading directory: {e}")
        return ""

    matches = []
    for filename in all_files:
        # match file file: csv or txt
        if not (filename.endswith('.csv') or filename.endswith('.txt')):
            continue
        # match place and type(sell or rent)
        filename_lower = filename.lower()
        if (data_type.lower() in filename_lower and 
            place.lower() in filename_lower):
            matches.append(filename)
    
    return matches[0] if matches else ""

# load .csv file to data[][]
data = [[pd.DataFrame() for _ in range(len(place))] for _ in range(len(data_type))]

for i in range(len(data_type)):
    for j in range(len(place)):
        # fill in the list
        file_name = get_file_name(data_type[i],place[j])
        data[i][j] = pd.read_csv(file_name)
        print(f"Loaded: {file_name} - Shape: {data[i][j].shape}")

        # check whether exist missing data
        print("Missing data check:")
        # if in a row, even just one is missing, the row will be deleted
        rows_with_any_missing = data[i][j].isnull().any(axis=1)
        num_incomplete_rows = rows_with_any_missing.sum()
        ratio_incomplete = num_incomplete_rows / len(data[i][j])
        # if there exist too many missing data
        if ratio_incomplete > 0.10:
            print(f"Warning: {ratio_incomplete:.1%} rows have missing data")
            print("Decision: Removing all incomplete rows despite high ratio, " 
                "as dataset only contains two critical columns (area and price/rent)")
            data[i][j] = data[i][j].dropna()
        elif num_incomplete_rows > 0:    # if just few missing data
            print(f"Info: {num_incomplete_rows} rows contain missing data")
            print("Action: Removing incomplete rows to ensure data integrity")
            data[i][j] = data[i][j].dropna()
        else:    # no data is missing
            print("Info: No missing data found")


### raw data description: without moving outlier

In [3]:
# data description:
def data_description(data):
    print(f"Sample size: {len(data)}")
    print(f"Mean: {data.mean():.2f}")
    print(f"Std:  {data.std():.2f}")
    print(f"Min:  {data.min():.2f}")
    print(f"25%:  {data.quantile(0.25):.2f}")
    print(f"50%:  {data.median():.2f}")
    print(f"75%:  {data.quantile(0.75):.2f}")
    print(f"Max:  {data.max():.2f}")

In [ ]:

for i in range(len(data_type)):
    for j in range(len(place)):
        current_type=data_type[i]
        current_place=place[j]
        # if data is rent, we need to calculate rent price per square
        # also, we make sure price column is the same name
        if current_type == "rent":
            data[i][j]['price_per_m2'] = data[i][j]['rent_yuan_per_month'] / data[i][j]['area_sqm']
        else:
            data[i][j]['price_per_m2'] = data[i][j]['unit_price_yuan_per_sqm']

        # house size data description:
        print(f"\n{place[j]} {data_type[i]} house size description:")
        data_description(data[i][j]['area_sqm'])
        
        # price_per_m2 data description:
        print(f"\n{place[j]} {data_type[i]} Price per m2 description:")
        data_description(data[i][j]['price_per_m2'])



### remove outlier
adopt 3-sigma method to define outlier and remove it.

In [ ]:
for i in range(len(data_type)):
    for j in range(len(place)):
        print(f"process outlier {data_type[i]}_{place[j]}")
        
        original_size = len(data[i][j])
        # use 3-sigma rule in area_sqm
        area_mean = data[i][j]['area_sqm'].mean()
        area_std = data[i][j]['area_sqm'].std()
        area_lower = area_mean - 3 * area_std
        area_upper = area_mean + 3 * area_std
        area_outliers = (data[i][j]['area_sqm'] < area_lower) | (data[i][j]['area_sqm'] > area_upper)

        # use 3-sigma rule in price_per_m2
        price_mean = data[i][j]['price_per_m2'].mean()
        price_std = data[i][j]['price_per_m2'].std()
        price_lower = price_mean - 3 * price_std
        price_upper = price_mean + 3 * price_std
        price_outliers = (data[i][j]['price_per_m2'] < price_lower) | (data[i][j]['price_per_m2'] > price_upper)

        # area or price is outlier -> remove
        outlier_mask = area_outliers | price_outliers
        data[i][j] = data[i][j][~outlier_mask]
        
        # report how many data is removed
        removed_count = original_size - len(data[i][j])
        print(f"Remaining data: {len(data[i][j])} rows")
        print(f"Removal percentage: {removed_count/original_size:.1%}")
        print()

### data description: after remove outlier

In [ ]:
for i in range(len(data_type)):
    for j in range(len(place)):
        current_place=place[j]
        current_type=data_type[i]

        # house size data description:
        print(f"\n{place[j]} {data_type[i]} house size description:")
        data_description(data[i][j]['area_sqm'])
        
        # price_per_m2 data description:
        print(f"\n{place[j]} {data_type[i]} Price per m2 description:")
        data_description(data[i][j]['price_per_m2'])

### calculate ratio

In [ ]:
ratios = []
for j in range(4):
    ratios.append(data[0][j]['price_per_m2'].median() / data[1][j]['price_per_m2'].median())

#### draw images

In [ ]:
def draw_ratio(ratios, title, place):
    plt.figure(figsize=(8, 5))
    bars = plt.bar(place, ratios)

    plt.axhline(y=200, color='red', linestyle='--', label='Global Fair Value (200)')

    plt.title(title)
    plt.xlabel('Block')
    plt.ylabel('Price to Rent Ratio')

    for bar, ratio in zip(bars, ratios):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, f'{ratio:.1f}', ha='center')

    plt.legend()
    plt.show()

In [ ]:
draw_ratio(ratios,'Figure A: Median Price to Rent Ratio for Each Block',place) 

## 3-4 Data Science Modeling (including plus model)

ordinary model: sell/rent_beta=[area_sqm,place[0],place[1],place[2],place[3]]
plus model: sell/rend_beta_plus=[area_sqm,area_sqm^2,place[0],place[1],place[2],place[3]

In [ ]:
# ----- SELL -----
sell_X      = np.empty((0,0))
sell_X_plus = np.empty((0,0))
sell_Y      = np.empty((0,1))
n = len(data[0])

for j in range(n):
    area = data[0][j]['area_sqm'].to_numpy().reshape(-1,1)    # convert vector to matrix
    area2 = area**2
    rows = area.shape[0]
    
    sell_dummy = np.tile([1 if k==j else 0 for k in range(n)], (rows,1))

    # basic model
    D = np.hstack([area, sell_dummy])    # because cannot connect empty matrix
    if sell_X.size == 0:
        sell_X = D
    else:
        sell_X = np.vstack([sell_X, D])

    # plus model
    D_plus = np.hstack([area, area2, sell_dummy])
    if sell_X_plus.size == 0:
        sell_X_plus = D_plus
    else:
        sell_X_plus = np.vstack([sell_X_plus, D_plus])

    # Y
    Y = data[0][j]['price_per_m2'].to_numpy().reshape(-1,1)
    if sell_Y.size == 0:
        sell_Y = Y
    else:
        sell_Y = np.vstack([sell_Y, Y])

# ----- RENT -----
rent_X      = np.empty((0,0))
rent_X_plus = np.empty((0,0))
rent_Y      = np.empty((0,1))
m = len(data[1])

for j in range(m):
    area = data[1][j]['area_sqm'].to_numpy().reshape(-1,1)
    area2 = area**2
    rows = area.shape[0]

    rent_dummy = np.tile([1 if k==j else 0 for k in range(m)], (rows,1))

    # basic model
    D = np.hstack([area, rent_dummy])
    if rent_X.size == 0:
        rent_X = D
    else:
        rent_X = np.vstack([rent_X, D])

    # plus model
    D_plus = np.hstack([area, area2, rent_dummy])
    if rent_X_plus.size == 0:
        rent_X_plus = D_plus
    else:
        rent_X_plus = np.vstack([rent_X_plus, D_plus])

    # Y
    Y = data[1][j]['price_per_m2'].to_numpy().reshape(-1,1)
    if rent_Y.size == 0:
        rent_Y = Y
    else:
        rent_Y = np.vstack([rent_Y, Y])


In [ ]:
# OLS: directly use result (X'X)^(-1)X'Y
sell_beta      = np.linalg.inv(sell_X.T @ sell_X) @ sell_X.T @ sell_Y
sell_beta_plus = np.linalg.inv(sell_X_plus.T @ sell_X_plus) @ sell_X_plus.T @ sell_Y

rent_beta      = np.linalg.inv(rent_X.T @ rent_X) @ rent_X.T @ rent_Y
rent_beta_plus = np.linalg.inv(rent_X_plus.T @ rent_X_plus) @ rent_X_plus.T @ rent_Y

print("sell beta:", sell_beta)
print("sell beta plus:", sell_beta_plus)
print("rent beta:", rent_beta)
print("rent beta plus:", rent_beta_plus)

## homework 3-5: comparison

1. according to econometrics theory, adding any independent variable (here is area^2), will always leads to higher R-square. Therefore, plus model of rent or sell has definitely higher R-square, but it's unknown whether plus model has higher adjusted R-square.

In [ ]:
# calculate fitted value
sell_hat      = (sell_X @ sell_beta).flatten()    # convert matrix to vector
rent_hat      = (rent_X @ rent_beta).flatten()
sell_hat_plus = (sell_X_plus @ sell_beta_plus).flatten()
rent_hat_plus = (rent_X_plus @ rent_beta_plus).flatten()

n = len(data[0])
ratios_hat = np.full((n,), np.nan, dtype=float)
ratios_hat_plus = np.full((n,), np.nan, dtype=float)
sell_counts = np.zeros((n,), dtype=int)
rent_counts = np.zeros((n,), dtype=int)

# data structure:
# rent/sell_X: [area, dummy_0, dummy_1, ...] -> dummy_offset = 1
# rent/sell_X_plus: [area, area^2, dummy_0, dummy_1, ...] -> dummy_offset_plus = 2

for j in range(n):
    # basic sell model
    sell_median = np.nan
    if j < n:
        col = 1 + j
        if col < sell_X.shape[1]:
            # 找到该 dummy 列为1的行索引（连续区间）
            idxs = np.where(sell_X[:, col] == 1)[0]
            if idxs.size > 0:
                start = idxs[0]
                end = idxs[-1] + 1  # 闭开区间 [start, end)
                vals = sell_hat[start:end]
                if vals.size > 0:
                    sell_median = np.median(vals)
                    sell_counts[j] = vals.size

    # basic rent model
    rent_median = np.nan
    if j < n:
        col_r = 1 + j
        if col_r < rent_X.shape[1]:
            idxs_r = np.where(rent_X[:, col_r] == 1)[0]
            if idxs_r.size > 0:
                start_r = idxs_r[0]
                end_r = idxs_r[-1] + 1
                vals_r = rent_hat[start_r:end_r]
                if vals_r.size > 0:
                    rent_median = np.median(vals_r)
                    rent_counts[j] = vals_r.size

    # calculate rations_hat
    if (not np.isnan(sell_median)) and (not np.isnan(rent_median)) and (rent_median != 0):
        ratios_hat[j] = sell_median / rent_median
    else:
        ratios_hat[j] = np.nan

    # plus model
    sell_median_p = np.nan
    if j < n:
        col_p = 2 + j
        if col_p < sell_X_plus.shape[1]:
            idxs_p = np.where(sell_X_plus[:, col_p] == 1)[0]
            if idxs_p.size > 0:
                start_p = idxs_p[0]
                end_p = idxs_p[-1] + 1
                vals_p = sell_hat_plus[start_p:end_p]
                if vals_p.size > 0:
                    sell_median_p = np.median(vals_p)

    rent_median_p = np.nan
    if j < n:
        col_rp = 2 + j
        if col_rp < rent_X_plus.shape[1]:
            idxs_rp = np.where(rent_X_plus[:, col_rp] == 1)[0]
            if idxs_rp.size > 0:
                start_rp = idxs_rp[0]
                end_rp = idxs_rp[-1] + 1
                vals_rp = rent_hat_plus[start_rp:end_rp]
                if vals_rp.size > 0:
                    rent_median_p = np.median(vals_rp)

    if (not np.isnan(sell_median_p)) and (not np.isnan(rent_median_p)) and (rent_median_p != 0):
        ratios_hat_plus[j] = sell_median_p / rent_median_p
    else:
        ratios_hat_plus[j] = np.nan


print("ratios_hat (base model)   :", ratios_hat)
print("ratios_hat_plus (plus model):", ratios_hat_plus)

In [ ]:
# draw Figure B
draw_ratio(ratios_hat,'Figure B: Median Price to Rent Ratio with Basic Model',place)

In [ ]:
# draw Figure C
draw_ratio(ratios_hat,'Figure C: Median Price to Rent Ratio with Plus Model',place)

In [ ]:
# error rete compareson
error_base = np.sum(np.abs(ratios - ratios_hat))
error_plus = np.sum(np.abs(ratios - ratios_hat_plus))

print("Total absolute error (base model) :", error_base)
print("Total absolute error (plus model) :", error_plus)

if error_base < error_plus:
    print("I prefer model 1/2 (base model)")
elif error_plus < error_base:
    print("I prefer model 1+/2+ (plus model)")
else:
    print("Both models perform equally")
